In [3]:
import pandas as pd
import numpy as np
csv_file_1 = 'water_quality.csv'
water = pd.read_csv(csv_file_1)


water = water.drop(['Measurement Timestamp Label', 'Measurement ID'], axis=1)
water['Measurement Timestamp'] = water['Measurement Timestamp'].astype(str).str[0:10].astype(np.str)
print(water['Beach Name'].unique())

water['Measurement Timestamp'] = pd.to_datetime(water['Measurement Timestamp'])
water = water.sort_values(by=['Measurement Timestamp'])

water.head()

['Montrose Beach' 'Ohio Street Beach' 'Calumet Beach' '63rd Street Beach'
 'Osterman Beach' 'Rainbow Beach']


,Beach Name,Measurement Timestamp,Water Temperature,Turbidity,Transducer Depth,Wave Height,Wave Period,Battery Life
0,Montrose Beach,2013-08-30,20.3,1.18,0.891,0.080,3.0,9.4
825,Osterman Beach,2013-08-31,21.5,3.51,1.538,0.231,4.0,9.4
2,Calumet Beach,2013-09-03,23.2,3.63,1.201,0.174,6.0,9.4
278,Ohio Street Beach,2013-09-03,21.9,4.97,1.039,0.241,7.0,9.4
823,63rd Street Beach,2013-09-18,18.9,7.56,1.517,0.140,4.0,11.0


In [25]:
water_group = water.groupby(['Beach Name','Measurement Timestamp'])['Turbidity','Water Temperature'].mean().reset_index()
water_group.head()

,Beach Name,Measurement Timestamp,Turbidity,Water Temperature
0,63rd Street Beach,2013-09-18,7.560000,18.900000
1,63rd Street Beach,2014-06-12,2.203333,17.766667
2,63rd Street Beach,2014-06-13,1.363333,16.758333
3,63rd Street Beach,2014-06-14,1.013750,17.195833
4,63rd Street Beach,2014-06-15,0.825833,17.087500


In [39]:
csv_file_1 = 'weather.csv'
weather = pd.read_csv(csv_file_1)
#weather = weather.drop(['Measurement Timestamp Label', 'Measurement ID'])
print(weather['Station Name'].unique())

weather = weather.drop(['Measurement Timestamp Label', 'Measurement ID','Battery Life','Heading','Solar Radiation','Barometric Pressure'], axis=1)
weather['Measurement Timestamp'] = weather['Measurement Timestamp'].astype(str).str[0:10].astype(np.str)


weather['Measurement Timestamp'] = pd.to_datetime(weather['Measurement Timestamp'])
weather = weather.sort_values(by=['Measurement Timestamp'])
weather.head()

['Oak Street Weather Station' 'Foster Weather Station'
 '63rd Street Weather Station']


,Station Name,Measurement Timestamp,Air Temperature,Wet Bulb Temperature,Humidity,Rain Intensity,Interval Rain,Total Rain,Precipitation Type,Wind Direction,Wind Speed,Maximum Wind Speed
1068,63rd Street Weather Station,2015-04-25,7.00,5.9,86.0,7.2,5.0,5.2,60.0,119.0,5.1,7.1
1285,63rd Street Weather Station,2015-04-30,6.10,4.3,76.0,0.0,0.0,2.5,0.0,11.0,7.2,13.0
0,Oak Street Weather Station,2015-05-22,NaN,7.0,55.0,0.0,0.0,1.4,0.0,63.0,1.9,2.8
205,Foster Weather Station,2015-05-22,9.89,NaN,52.0,NaN,0.0,NaN,NaN,61.0,2.2,2.5
225,Foster Weather Station,2015-05-22,9.28,NaN,61.0,NaN,0.0,NaN,NaN,40.0,1.2,1.7


In [40]:
weather_group = weather.groupby(['Station Name','Measurement Timestamp'])['Air Temperature'].mean().reset_index(name = 'Mean Air Temp')
weather_group.head()

,Station Name,Measurement Timestamp,Mean Air Temp
0,63rd Street Weather Station,2015-04-25,7.000000
1,63rd Street Weather Station,2015-04-30,6.100000
2,63rd Street Weather Station,2015-06-22,27.300000
3,63rd Street Weather Station,2015-06-23,20.333333
4,63rd Street Weather Station,2015-06-24,21.450000


In [57]:
weather_group.rename(columns={'Station Name': 'Beach Name'}, inplace=True)
weather_group = weather_group.replace(to_replace ='Oak Street Weather Station',  value ='Ohio Street Beach')
#the reasons we called Ohio Street Beach because Ohio Street Beach is closed by oak street
weather_group = weather_group.replace(to_replace ='Foster Weather Station',  value ='Montrose Beach')
#the reasons we called Ohio Street Beach because Montrose beach is closed by Ohio street
weather_group = weather_group.replace(to_replace ='63rd Street Weather Station',  value ='63rd Street Beach')

weather_group.head()



,Beach Name,Measurement Timestamp,Mean Air Temp
0,63rd Street Beach,2015-04-25,7.000000
1,63rd Street Beach,2015-04-30,6.100000
2,63rd Street Beach,2015-06-22,27.300000
3,63rd Street Beach,2015-06-23,20.333333
4,63rd Street Beach,2015-06-24,21.450000


In [68]:
water_air = pd.merge(weather_group, water_group,  how='left', on = ['Beach Name', 'Measurement Timestamp'])
water_air = water_air.dropna()
water_air.head()

,Beach Name,Measurement Timestamp,Mean Air Temp,Turbidity,Water Temperature
2,63rd Street Beach,2015-06-22,27.300000,1.128095,17.471429
3,63rd Street Beach,2015-06-23,20.333333,1.404444,17.800000
4,63rd Street Beach,2015-06-24,21.450000,1.258000,18.335000
5,63rd Street Beach,2015-06-25,20.104348,0.836667,17.938095
6,63rd Street Beach,2015-06-26,16.708333,1.100000,17.410526


In [21]:
import json
from pandas.io.json import json_normalize

with open('ecoli.json') as data_file:
    data = json.load(data_file)

quality = json_normalize(data,'data')



#json_file = open('rows.json')
#json_str = json_file.read()
#json_data = json.loads(json_str)
#type(json_data)
#openfile=open(json_file)
#jsondata=json.load(openfile)
#df=pd.DataFrame(jsondata)

#ecoli = pd.read_json(json_file, lines = True)

quality = quality[quality.columns[9:14]]
quality = quality.rename(index = str, columns = {9:"Timestamp", 10: "Beach", 11:"Sample 1", 12:"Sample 2", 13:"Average"} )
quality['Timestamp'] = quality['Timestamp'].astype(str).str[0:10].astype(np.str)
quality['Average'] = quality['Average'].astype(float)
print(quality['Beach'].unique())

quality['Timestamp'] = pd.to_datetime(quality['Timestamp'],errors='coerce')
quality = quality.sort_values(by=['Timestamp'])


quality.head()

['12th Street' '57th Street' '63rd Street' 'Calumet' 'Foster'
 'Hartigan (Albion)' 'Howard' 'Humboldt' 'Leone'
 'Margaret T Burroughs (31st)' 'Marion Mahony Griffin (Jarvis)' 'Montrose'
 'North Avenue' 'Oak Street' 'Oakwood' 'Ohio Street' 'Osterman' 'Rainbow'
 'Rogers' 'South Shore' 'Margaret T Burroughs' 'Hartigan'
 'Marion Mahony Griffin' 'Juneway' 'Fargo' 'Montrose Dog' 'NA' 'Columbia'
 'Lane' 'Loyola' 'North Shore']


,Timestamp,Beach,Sample 1,Sample 2,Average
4908,2015-05-26,South Shore,77.451082547,280.5556268,147.0
4904,2015-05-26,63rd Street,241.37120153,69.978080303,130.0
4907,2015-05-26,Rainbow,212.79680018,117.1123661,158.0
4906,2015-05-26,Montrose,60.313963186,149.55082696,95.0
4905,2015-05-26,Calumet,392.76038571,111.08459692,209.0


In [71]:
quality_group = quality.groupby(['Beach','Timestamp'])['Average'].mean().reset_index()
quality_group.rename(columns={'Beach': 'Beach Name'}, inplace=True)
quality_group.rename(columns={'Timestamp': 'Measurement Timestamp'}, inplace=True)
quality_group.rename(columns={'Average': 'EColi Sample PPM'}, inplace=True)
quality_group = quality_group.replace(to_replace ='Foster',  value ='Montrose Beach')
quality_group = quality_group.replace(to_replace ='Oak Street',  value ='Ohio Street Beach')
quality_group = quality_group.replace(to_replace ='63rd Street',  value ='63rd Street Beach')

quality_group = quality_group.dropna()
quality_group.head(100)

,Beach Name,Measurement Timestamp,EColi Sample PPM
0,12th Street,2016-05-26,130.230565
1,12th Street,2016-06-01,115.568162
2,12th Street,2016-06-02,74.833148
3,12th Street,2016-06-03,344.949272
4,12th Street,2016-06-04,93.893557
5,12th Street,2016-06-05,16.431677
6,12th Street,2016-06-08,563.361341
7,12th Street,2016-06-09,59.380131
8,12th Street,2016-06-10,486.518242
9,12th Street,2016-06-11,61.846584


In [74]:
merged = pd.merge(water_air, quality_group,  how='left', on = ['Beach Name', 'Measurement Timestamp'])
merged = merged.dropna()
merged.head(100)

,Beach Name,Measurement Timestamp,Mean Air Temp,Turbidity,Water Temperature,EColi Sample PPM
1,63rd Street Beach,2015-06-23,20.333333,1.404444,17.800000,33.000000
2,63rd Street Beach,2015-06-24,21.450000,1.258000,18.335000,211.000000
3,63rd Street Beach,2015-06-25,20.104348,0.836667,17.938095,1878.000000
4,63rd Street Beach,2015-06-26,16.708333,1.100000,17.410526,28.000000
8,63rd Street Beach,2015-06-30,18.945833,0.809565,18.886957,423.000000
9,63rd Street Beach,2015-07-01,14.871429,1.707778,18.761111,22.000000
10,63rd Street Beach,2015-07-02,16.150000,3.435714,19.314286,49.000000
15,63rd Street Beach,2015-07-07,17.150000,7.878824,18.294118,1201.000000
16,63rd Street Beach,2015-07-08,16.312500,10.788125,17.906250,189.000000
17,63rd Street Beach,2015-07-09,17.400000,7.037222,18.611111,63.000000
